In [1]:
import pandas as pd
from backtesting_functions import quarters_dict
from datetime import datetime, timedelta
from sklearn.cluster import KMeans
import numpy as np
from sklearn.model_selection import train_test_split
import backtesting_functions

In [8]:
quarters_dict = {
    "2021_Q1": "2021-04-21",
    "2021_Q2": "2021-07-21",
    "2021_Q3": "2021-10-21",
    "2021_Q4": "2022-01-21",
    "2022_Q1": "2022-04-21",
    "2022_Q2": "2022-07-21",
    "2022_Q3": "2022-10-21",
    "2022_Q4": "2023-01-21",
    "2023_Q1": "2023-04-21",
    "2023_Q2": "2023-07-21",
    "2023_Q3": "2023-10-21",
    "2023_Q4": "2024-01-21",
    "2024_Q1": "2024-04-21",
    "2024_Q2": "2024-07-21",
    "2024_Q3": "2024-10-21"
}


In [2]:
# load data
idx = []
for year in [2021, 2022, 2023, 2024]:
    for quarter in ['Q1', 'Q2', 'Q3', 'Q4']:
        idx.append(f"{year}_{quarter}")

df_list = [pd.read_csv(f'../../data/processed/quarterly/{id}.csv') for id in idx]
cols = df_list[0].columns[3:]

# train models
models = {}
for i, (dataframe, quarter) in enumerate(zip(df_list, idx)):
    kmeans = KMeans(n_clusters=15, random_state=42, n_init=10)
    kmeans.fit(dataframe[cols])

    models[quarter] = kmeans

df_dict = {quarter: dataframe for quarter, dataframe in zip(idx, df_list)}

prices = pd.read_csv('prices.csv')

In [3]:
def get_data(symbol, quarter):
    df = df_dict[quarter]
    row = df[df['symbol'] == symbol]

    if row.empty:
        return None
    
    cols = row.columns[3:]
    row_filtered = row[cols]

    return row_filtered

def returns(price_df, days=14):
    """Calculate ratio between start of period MA and end of period MA"""
    
    # 1day to 1day returns could be sensitive to market noise, calculate average instead
    start_period = price_df.iloc[0:days]
    end_period = price_df.iloc[-days:-1]

    start_period_avg = start_period.mean()
    end_period_avg = end_period.mean()
    
    returns_df = ((end_period_avg - start_period_avg) / start_period_avg).reset_index()
    returns_df.columns = ['symbol', 'returns']

    return returns_df

def calculate_relative_performance(prices, cluster, train_quarter):
   train_date = quarters_dict[train_quarter]
   test_date = str(datetime.strptime(train_date, '%Y-%m-%d') + timedelta(days=90)).split(' ')[0]
   val_date = str(datetime.strptime(test_date, '%Y-%m-%d') + timedelta(days=90)).split(' ')[0]
   
   cluster_prices = prices[cluster]
   cluster_prices = cluster_prices.loc[test_date:val_date]
   
   returns_df = returns(cluster_prices, days=7)

   results = []
   for stock in cluster:

      stock_return = returns_df[returns_df['symbol'] == stock]['returns'].values[0]
      cluster_avg_return = returns_df[returns_df['symbol'] != stock]['returns'].mean()
      relative_performance = stock_return - cluster_avg_return
      results.append({'symbol': stock, 'relative_performance': relative_performance})

   return pd.DataFrame(results)

In [4]:
def get_cluster_mapping(model, quarter, train_stocks):
    cluster_stock_mapping = {}
    for stock in train_stocks:
        clustering_data = get_data(stock, quarter)
        if clustering_data is not None:
            cluster = model.predict(clustering_data)[0]
            if cluster not in cluster_stock_mapping:
                cluster_stock_mapping[cluster] = []
            cluster_stock_mapping[cluster].append(stock)
    return cluster_stock_mapping

In [62]:
def next_quarter(current_quarter):
    year, quarter = current_quarter.split('_')
    year = int(year)
    quarter_number = int(quarter[1])
    
    if quarter_number == 4:
        return f"{year + 1}_Q1"
    else:
        return f"{year}_Q{quarter_number + 1}"

def construct_data(cluster, quarter):
    cluster_data = {symbol: get_data(symbol, quarter) for symbol in cluster}
    cluster_df = pd.concat(cluster_data, axis=0).reset_index(level=0).rename(columns={'level_0': 'symbol'})
    return cluster_df

def centroid(cluster_data):
    cols = cluster_data.columns[1:]
    centroid_values = cluster_data[cols].mean()
    centroid_df = pd.DataFrame([centroid_values], columns=cols)
    centroid_df.insert(0, 'symbol', 'centroid')
    return centroid_df

def center_distance(cluster_data):
    cols = cluster_data.columns[1:]
    centroid_df = centroid(cluster_data)
    cluster_data_with_centroid = pd.concat([cluster_data, centroid_df], ignore_index=True)
    delta = cluster_data_with_centroid[cols].sub(centroid_df.iloc[0][cols], axis=1)
    cluster_data_with_centroid[cols] = delta
    return cluster_data_with_centroid

def construct_params(prices, cluster, quarter):
    cluster_data_0 = construct_data(cluster, quarter)
    cluster_data_1 = construct_data(cluster, next_quarter(quarter))

    dist_0 = center_distance(cluster_data_0)
    dist_1 = center_distance(cluster_data_1)

    relative_performance = calculate_relative_performance(prices, cluster, quarter)
    symbols = dist_0[['symbol']].copy()

    cols = dist_0.columns.drop('symbol')
    delta_values = dist_1[cols] - dist_0[cols]
    delta_values.columns = [f"{col}_1" for col in delta_values.columns]

    delta_dist = pd.concat([symbols, delta_values], axis=1)

    merged_params = dist_0.merge(delta_dist, on='symbol', suffixes=('_0', '_1'))
    final_params = merged_params.merge(relative_performance, on='symbol')

    return final_params

def construct_params(prices, cluster, quarter):
    # Get cluster data for quarter and next quarter
    cluster_data_0 = construct_data(cluster, quarter)
    cluster_data_1 = construct_data(cluster, next_quarter(quarter))

    # Compute relative distance from centroid
    dist_0 = center_distance(cluster_data_0)
    dist_1 = center_distance(cluster_data_1)

    # Compute delta (movement relative to cluster)
    symbols = dist_0[['symbol']].copy()
    feature_cols = dist_0.columns.drop('symbol')
    
    delta = dist_1[feature_cols] - dist_0[feature_cols]
    delta.columns = [f"{col}_delta" for col in feature_cols]

    # Get relative performance at t1
    relative_perf = calculate_relative_performance(prices, cluster, next_quarter(quarter))

    # Get relative performacne at t2
    relative_perf_t2 = calculate_relative_performance(prices, cluster, next_quarter(next_quarter(quarter)))
    relative_perf_t2.columns = ['symbol', 'target']

    # Combine everything
    df = pd.concat([symbols, dist_0[feature_cols], delta], axis=1)
    df = df.merge(relative_perf, on='symbol')
    df = df.merge(relative_perf_t2, on='symbol')

    return df


In [18]:
def get_cluster_mapping(model, quarter, stock_list):
    cluster_stock_mapping = {}
    for stock in stock_list:
        clustering_data = get_data(stock, quarter)
        if clustering_data is not None:
            cluster = model.predict(clustering_data)[0]
            if cluster not in cluster_stock_mapping:
                cluster_stock_mapping[cluster] = []
            cluster_stock_mapping[cluster].append(stock)
    return cluster_stock_mapping

In [63]:
# construct agg. dataframe
from sklearn.model_selection import train_test_split

all_symbols = list(set(symbol for df in df_list for symbol in df['symbol']))

# Train-test split
train_stocks, test_stocks = train_test_split(all_symbols, train_size=0.8, random_state=17)

# Load prices
prices = pd.read_csv('prices.csv')
prices['Date'] = pd.to_datetime(prices['Date'])
prices.set_index('Date', inplace=True)

def build_dataset(stock_list):
    data_dict = {}
    for quarter in list(quarters_dict.keys())[:-2]:
        cluster_stock_mapping = get_cluster_mapping(models[quarter], quarter, all_symbols)  # Use all_symbols here

        df_list = []
        for cluster in cluster_stock_mapping.values():
            valid_cluster = [symbol for symbol in cluster if symbol in prices.columns]
            filtered_cluster = [symbol for symbol in valid_cluster if symbol in stock_list]
            if filtered_cluster:
                df = construct_params(prices, filtered_cluster, quarter)
                df_list.append(df)

        data_dict[quarter] = pd.concat(df_list, ignore_index=True)
    return data_dict

# Build datasets
train_data_dict = build_dataset(train_stocks)
test_data_dict = build_dataset(test_stocks)

# Combine across all quarters
train_df = pd.concat(train_data_dict.values(), ignore_index=True)
test_df = pd.concat(test_data_dict.values(), ignore_index=True)


In [64]:
train_df

,symbol,currentRatio,quickRatio,returnOnEquity,returnOnAssets,netProfitMargin,priceEarningsRatio,priceBookValueRatio,priceToSalesRatio,freeCashFlowPerShare,...,priceToSalesRatio_delta,freeCashFlowPerShare_delta,operatingCashFlowPerShare_delta,cashFlowToDebtRatio_delta,debtEquityRatio_delta,longTermDebtToCapitalization_delta,assetTurnover_delta,inventoryTurnover_delta,relative_performance,target
0,USB,0.864497,0.839717,0.00718,-0.124411,0.654929,-0.168055,-0.036306,0.263033,-0.104637,...,-0.059029,0.370028,0.401834,0.139153,0.016288,-0.112291,0.014989,-1.016134,-0.005716,-0.070382
1,PRU,-2.461608,-2.296548,0.097067,-0.173017,-0.702726,-1.012081,-0.481575,-1.114998,-0.565991,...,0.044446,1.911021,2.112591,1.27341,-0.030164,-0.132096,-0.006522,-1.016134,0.019616,0.083064
2,SCHW,1.620406,1.552482,-0.336292,-0.19645,0.125921,0.562742,0.157639,0.687692,-0.565991,...,0.103223,-0.960168,-0.983382,-0.790462,0.037861,-0.058835,0.008542,-1.016134,0.128669,-0.087540
3,JPM,-1.453974,-1.346425,0.140089,-0.136236,0.774305,-0.259272,-0.014397,0.224566,-0.565991,...,0.020626,0.989325,1.118735,-0.487902,0.067828,-0.02945,0.008467,-1.016134,-0.020093,-0.096089
4,C,-0.452408,-0.402025,-0.088997,-0.157761,0.580074,-0.706031,-0.411557,-0.260287,1.778225,...,0.006534,-3.304384,-3.595776,-1.712963,0.079529,-0.041972,0.005406,-1.016134,-0.070208,-0.193754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4940,VRSK,0.454992,0.487129,1.914648,0.596169,-0.13137,0.574416,0.827621,0.416299,0.045642,...,0.034991,-0.333095,-0.281168,-0.379874,-0.925328,-0.135871,-0.077951,11.712511,-0.000627,NaN
4941,AMT,-0.164793,-0.176432,-5.987218,-1.271134,-0.05635,0.125256,-0.696321,0.082967,-0.045968,...,-0.078134,0.208507,0.174695,0.185038,-0.294126,0.003217,0.049382,-6.295934,-0.329992,NaN
4942,MPWR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4943,CL,0.130423,-0.114826,-2.446589,0.995392,0.325794,-0.558409,-0.638395,-0.034497,0.231834,...,-0.006563,0.182535,-0.145397,0.131272,6.342383,0.001179,-0.068099,-8.868835,0.108436,NaN


Fit linear regression to the data

In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Drop rows with NA values
train_df_clean = train_df.dropna()
test_df_clean = test_df.dropna()

# Separate features (X) and target (y)
X_train = train_df_clean.drop(columns=['symbol', 'target'])
y_train = train_df_clean['target']

X_test = test_df_clean.drop(columns=['symbol', 'target'])
y_test = test_df_clean['target']

# Fit linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict on test data
y_pred = lr.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.019910950563342186
